In [1]:
#!pip install autocorrect
#!pip install nltk
#!pip install textblob
#!pip install vaderSentiment

In [2]:
import json
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline  

import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from autocorrect import Speller 
from nltk.stem import LancasterStemmer, WordNetLemmatizer, PorterStemmer
#from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score



In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\10686126\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:


with open('TextCase.json') as f:
  data = json.load(f)


In [5]:
type(data)
print(data.keys())

dict_keys(['Specificity', 'ModelsInfo', 'JobInfo', 'Data'])


In [6]:
print(data["Specificity"])
print(data["ModelsInfo"])
print(data["JobInfo"])

{'Entity': 'Amazon', 'Theme': 'Books', 'Atomic': None, 'Language': 'English'}
{'KNN': False, 'SVM': True, 'RF': True, 'LR': True, 'BL': False}
{'ID': 10001, 'Client': 'Elsevier'}


In [7]:
Comment_data=pd.DataFrame(data["Data"])
print(Comment_data.shape)
Comment_data = Comment_data.iloc[1:]#removing 1st row as not of much use but can cause some algo to fail
Comment_data.head(5)

(91681, 3)


,CommentID,CommentText,Category
1,2,accenture is a fine company however the pepe...,Neutral
2,3,it has become a laggard in remuneration benef...,Neutral
3,4,like myself most people i know do not necessa...,Neutral
4,5,no action on career development plans.,Positive
5,6,no visible adavancement pathway.,Neutral


In [8]:
Comment_data.isna().sum()

CommentID      0
CommentText    0
Category       0
dtype: int64

In [9]:
X=Comment_data["CommentText"]
y=Comment_data["Category"]

In [10]:
y.unique()

array(['Neutral', 'Positive', 'Negative'], dtype=object)

In [11]:
## lowercasing
X=X.str.lower()
## remove punctuation

# def remove_punctuation(text):
#      final = "".join(u for u in text if u not in ("?", ".", ";", ":",  "!",'"'))
#     return final
# x= X.apply(remove_punctuation)
X= X.str.replace('[^\w\s]','')
# stopword removal

stop = stopwords.words('english')
# saving sentiment affecting words
wanted_words=['no', 'nor', 'not', "don't","isn't"]+(stop[stop.index("aren't"):])
stopword_final = [ele for ele in stop if ele not in wanted_words]
#print(stopword_final)

X = X.apply(lambda x: " ".join(x for x in x.split() if x not in stop))

#stemming
st = PorterStemmer()
X = X.apply(lambda x: " ".join([st.stem(word) for word in x.split()]))
print(X)
#autocorrecting misspell words

# spell = Speller(lang='en')# takes some time .
# X=X.apply(lambda x: " ".join([spell(i) for i in x.split()]))

1        accentur fine compani howev pepetu quest const...
2        becom laggard remuner benefit promot capabl re...
3        like peopl know necessarili want leav compani ...
4                               action career develop plan
5                                   visibl adavanc pathway
                               ...                        
91676              would like sustain longer time accentur
91677           safe work environ teamwork forc rel strong
91678    think accentur support employe initi stage get...
91679                    reward recognit motiv work effici
91680                            look great futur accentur
Name: CommentText, Length: 91680, dtype: object


# Solving using library



In [12]:

def sentiment_scores(sentence):
 
    # Create a SentimentIntensityAnalyzer object.
    sid_obj = SentimentIntensityAnalyzer()
 
    # polarity_scores method of SentimentIntensityAnalyzer
    # object gives a sentiment dictionary.
    # which contains pos, neg, neu, and compound scores.
    sentiment_dict = sid_obj.polarity_scores(sentence)
     
    # decide sentiment as positive, negative and neutral
    if sentiment_dict['compound'] >= 0.05 :
        return "Positive"
 
    elif sentiment_dict['compound'] <= - 0.05 :
        return"Negative"
 
    else :
        return"Neutral"
 
 

In [ ]:
Y = X.apply(sentiment_scores)
Y

In [ ]:
confusion_matrix(y, Y)
accuracy_score(y,Y)

# Training a machine learning model 